In [1]:
import numpy as np
from itertools import combinations
from copy import deepcopy
from functools import cache

In [2]:
test = ['???.### 1,1,3',
        '.??..??...?##. 1,1,3',
        '?#?#?#?#?#?#?#? 1,3,1,6',
        '????.#...#... 4,1,1',
        '????.######..#####. 1,6,5',
        '?###???????? 3,2,1']

data = np.genfromtxt('day12_input.txt', dtype=str, delimiter='\n', comments=None)

In [3]:
def parse(data):
    new_data = []
    for line in data:
        springs, pos = line.split()
        pos_n = pos.split(',')
        poses = []
        for p in pos_n:
            poses.append(int(p))
        new_data.append([springs, np.array(poses)])
    return new_data

def find_springs(data):
    poss_springs = []
    count = 0
    for line in data:
        target, groups = line
        #print(target)
        can_be = []
        
        t_dots = np.where(np.array(list(target)) == '.')[0]
        t_hasz = np.where(np.array(list(target)) == '#')[0]
        
        n_groups = len(groups)
        n_empty = len(target)-(np.sum(groups)+n_groups-1)

        #print(n_groups, n_empty, len(target))
        rng = n_groups+n_empty
        #print(range(rng))

        opts = combinations(range(rng), n_groups)
        
        for op in opts:
            #print(op)
            string = ''
            i = 0
            j = 0
            while len(string) < len(target):
                if j < len(op) and i == op[j]:
                    for k in range(0, groups[j]):
                        string += '#'
                    if j < len(op)-1:
                        string += '.'
                    j += 1
                else:
                    string += '.'
                i += 1

            reject = False
            s_dots = np.where(np.array(list(string)) == '.')[0]
            for td in t_dots:
                if td not in s_dots:
                    reject = True
                    break
            if reject:
                #print('rejecting', string, '.')
                continue

            s_hasz = np.where(np.array(list(string)) == '#')[0]
            for th in t_hasz:
                if th not in s_hasz:
                    reject = True
                    break
            if reject:
                #print('rejecting', string, '#')
                continue
                
            can_be.append(string)
            count += 1
            
        #for cb in can_be:
        #    print(cb)
        #print(len(can_be))
        #print()
        #poss_springs.append(can_be)
        #print(target, len(can_be))
        
    return count#, poss_springs

test_parse = parse(test)
test_count = find_springs(test_parse)
print(test_count)
print()
data_parse = parse(data)
data_count = find_springs(data_parse)
print('Part 1 result:', data_count)

21

Part 1 result: 7169


In [5]:
@cache
def count_arrangements(springs, groups, i, j):
    if j == len(groups):
        return 1
    #print('depth:', j, '\t', i, groups[j])
    if i > len(springs):
        return 0
    
    if memo[i][j] != -1:
        return memo[i][j]
        
    is_spaca = i+groups[j] <= len(springs)
    if is_spaca == False:
        return 0
    
    hasz_dot = springs[i] == '#' and '.' in springs[i+1:i+groups[j]]
    if hasz_dot:
        return 0
    
    can_hasz = np.sum(groups[:j])
    num_hasz = springs[:i].count('#')
    if num_hasz > can_hasz:
        return 0
    
    boing = list(springs)
    for s in range(i,i+groups[j]):
        boing[s] = '#'
    boing = ''.join(boing)
    can_place = True
    if boing.count('#') > np.sum(groups):
        can_place = False
    
    is_spacb = '.' not in springs[i:i+groups[j]]
    is_possa = (i == 0 or springs[i-1] != '#')
    is_possb = False
    if i+groups[j] > len(springs):
        is_possb = False
    elif i+groups[j] == len(springs):
        is_possb = True
    elif springs[i+groups[j]] != '#':
        is_possb = True
    
    result = 0
        
    #if i == 0 and j == 0:
    #    print(i+groups[j] <= len(springs), is_spaca, is_spacb, is_possa, is_possb)
    
    if can_place and is_spacb and is_possa and is_possb:
       # print(i, j, groups[j])
        next_springs = deepcopy(springs)
        next_springs = list(next_springs)
        for s in range(i,i+groups[j]):
            next_springs[s] = '#'
        next_springs = ''.join(next_springs)
        
        result += count_arrangements(next_springs, groups, i+groups[j]+1, j+1)
        #if i == 0 and j == 0:
        #    print('\t', result)
        
    result += count_arrangements(springs, groups, i+1, j)
    
    memo[i][j] = result
    return result

def find_more_springs(data, part=1):
    data = parse(data)
    
    total = 0
    global memo
    for line in data:
        target, groups = line
        
        if part == 2:
            target = '?'.join([target]*5)
            groups = np.tile(groups,5)
            
        memo = np.ones((len(target)+1,len(groups)+1), dtype=int)*-1
        result = count_arrangements(target, tuple(groups), 0, 0)
        #print(target, result)
        total += result
    return total
#print(total)
    
print(find_more_springs(test))
print()
print('Part 1 result:', find_more_springs(data))

21

Part 1 result: 7169


In [6]:
print(find_more_springs(test, 2))
print()
print('Part 2 result:', find_more_springs(data, 2))

525152

Part 2 result: 1738259948652
